<a href="https://colab.research.google.com/github/Baroka-wp/dive_ML/blob/master/Sprint8_apprentissage_d'ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn import datasets

### Préparation d'un petit ensemble de données


In [8]:
# importer les donnees
data=pd.read_csv('/content/train.csv').select_dtypes(include='number')

In [9]:
# supprimer les donnees null
data.isnull().sum()
data = data.fillna(data.mean())

# definir les variables X,y
X = data[['GrLivArea','SalePrice']].values
y = data['SalePrice'].values

X = np.log1p(X)
y = np.log1p(y)

print('X :{}, y :{}'.format(X.shape,y.shape))

X :(1460, 2), y :(1460,)


####Veuillez diviser train.csv en 80% pour la formation (train) et 20% pour la vérification (val).




In [10]:

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=1)
print('X_train :{}, y_test:{}'.format(X_train.shape,y_test.shape))

X_train :(1168, 2), y_test:(292,)


## [Problem 1] Blending scratch mounting

#### Méthode (par exemple, régression linéaire, SVM, arbre de décision, réseau de neurones, etc.)


In [17]:
models = [LinearRegression(),SVR(),DecisionTreeRegressor()]
predict = list()

for model in models:
    model.fit(X_train,y_train)
    predict.append(model.predict(X_test))
    
predict_array = np.array(predict)
blending = np.mean(predict_array,axis=0)

print('MSE blend:{:.5f}'.format(mean_squared_error(y_test,blending)))

MSE blend:0.00022


#### Hyper paramètres (par exemple type de noyau SVM, valeurs de poids initiales, etc.)

In [12]:
poids = [1,5,10]
SVR_predicts = list()
for c in poids:
  svr = SVR(C=c)
  svr.fit(X_train,y_train)
  svr_predict = svr.predict(X_test)
  SVR_predicts.append(svr_predict)


SVR_predicts_array = np.array(SVR_predicts)
SVR_blend = np.mean(SVR_predicts_array, axis=0)

print('MSE blend:{:.5f}'.format(mean_squared_error(y_test,SVR_blend)))



MSE blend:0.00220


#### prétraiter les données d'entrée (par exemple, normalisation, conversion logarithmique, PCA, etc.)


In [18]:
#pretraitement
pca = PCA(n_components = 2)
pca.fit(X_train)
X_train_trans = pca.transform(X_train)
X_test_trans = pca.transform(X_test)

In [21]:
models_2 = [LinearRegression(),SVR(),DecisionTreeRegressor()]
predict_2 = list()

for model in models_2:
    model.fit(X_train_trans,y_train)
    predict_2.append(model.predict(X_test_trans))
    
predict_array_2 = np.array(predict_2)
blending_2 = np.mean(predict_array_2,axis=0)

print('MSE blend:{:.5f}'.format(mean_squared_error(y_test,blending_2)))

MSE blend:0.00067


## [Problem 2] Scratch mounting of bagging


In [20]:
X_train_bagging, X_test_bagging, y_train_bagging, y_test_bagging = train_test_split(X,y,test_size=0.2,shuffle=True)


In [22]:
models_bagging = [LinearRegression(),SVR(),DecisionTreeRegressor()]
predict_bagging = list()

for model in models_bagging:
    model.fit(X_train_bagging,y_train_bagging)
    predict_bagging.append(model.predict(X_test_bagging))
    
predict_array_bagging = np.array(predict_bagging)
bagging = np.mean(predict_array_bagging,axis=0)

print('MSE blend:{:.5f}'.format(mean_squared_error(y_test_bagging,bagging)))

MSE blend:0.00020


## [Problem 3] Stacking scratch mounting


In [23]:
# sample data
X, y = datasets.make_classification(n_samples=10000, n_features=20, n_informative=15, n_redundant=5, random_state=7)


In [24]:
# split data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.5,random_state=1)

In [32]:
# diviser les donnee pour la validations 
X_train_1, X_val, y_train_1, y_val = train_test_split(X_train,y_train,test_size=0.2,random_state=1)


In [31]:
# selectionner l'ensemble de models
models_stacking = [LinearRegression(), KNeighborsClassifier(), DecisionTreeRegressor()]
predictions = list()

for model in models_stacking:
   model.fit(X_train_1, y_train_1)
   y_pred = model.predict(X_val)
   predictions.append(y_pred)

predictions = np.array(predictions)
new_X = np.mean(predictions,axis=0).reshape(-1,1)

#choisir le model de reference
blender = LinearRegression()
blender.fit(new_X, y_val)

blender_pred = blender.predict(new_X) 
print("Blender mse:{:.5f}".format(mean_squared_error(blender_pred,y_val)))


Blender mse:0.04423


In [33]:
# comparaison avec le model 
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)
y_pred_lr= model_lr.predict(X_test)
print("Logistic regression:{:.3f}".format(mean_squared_error(y_test,y_pred_lr)))

Logistic regression:0.110
